In [ ]:
! pip install https://github.com/elyase/geotext/archive/master.zip
! pip install jsonlines

In [53]:
import jsonlines
import ast
import json
from geotext import GeoText
import collections

In [2]:
# https://stackoverflow.com/a/36599122

a


The data has a lot of issues. It cannot be directly read by Spark (you get a 'corrupted record') as there is a lot of nesting. Similarly, you get an error when you try to read the data with pandas. Python's native json library and the jsonlines module cannot read it because it contains single quotes rather than double quotes (json requires double quotes). The solution is to read the data line by line, use the ast library to convert single quotes to double quotes, remove trailing commas, and dump it to valid json

In [6]:
json_data = "{'property': 'text'}"
json_data2 = ast.literal_eval(json_data)
print(json.dumps(json_data2))

{"property": "text"}


In [9]:
with open('first13lines.jsonl', 'r') as testfile:
    for line in testfile:
        print(json.dumps(line))

"{'name': u'Diamond Valley Lake Marina', 'price': None, 'address': [u'2615 Angler Ave', u'Hemet, CA 92545'], 'hours': [[u'Monday', [[u'6:30 am--4:15 pm']]], [u'Tuesday', [[u'6:30 am--4:15 pm']]], [u'Wednesday', [[u'6:30 am--4:15 pm']], 1], [u'Thursday', [[u'6:30 am--4:15 pm']]], [u'Friday', [[u'6:30 am--4:15 pm']]], [u'Saturday', [[u'6:30 am--4:15 pm']]], [u'Sunday', [[u'6:30 am--4:15 pm']]]], 'phone': u'(951) 926-7201', 'closed': False, 'gPlusPlaceId': '104699454385822125632', 'gps': [33.703804, -117.003209]}\n"
"{'name': u'Blue Ribbon Cleaners', 'price': None, 'address': [u'Parole', u'Annapolis, MD'], 'hours': None, 'phone': u'(410) 266-6123', 'closed': False, 'gPlusPlaceId': '103054478949000078829', 'gps': [38.979759, -76.547538]}\n"
"{'name': u'Portofino', 'price': None, 'address': [u'\\u0443\\u043b. \\u0422\\u0443\\u0442\\u0430\\u0435\\u0432\\u0430, 1', u'Nazran, Ingushetia, Russia', u'366720'], 'hours': [[u'Monday', [[u'9:30 am--9:00 pm']]], [u'Tuesday', [[u'9:30 am--9:00 pm']]],

In [8]:
# Test literal_eval
with open('first13lines.jsonl', 'r') as testfile:
    for line in testfile:
        line2 = ast.literal_eval(line)
        print(json.dumps(line2))
        

{"name": "Diamond Valley Lake Marina", "price": null, "address": ["2615 Angler Ave", "Hemet, CA 92545"], "hours": [["Monday", [["6:30 am--4:15 pm"]]], ["Tuesday", [["6:30 am--4:15 pm"]]], ["Wednesday", [["6:30 am--4:15 pm"]], 1], ["Thursday", [["6:30 am--4:15 pm"]]], ["Friday", [["6:30 am--4:15 pm"]]], ["Saturday", [["6:30 am--4:15 pm"]]], ["Sunday", [["6:30 am--4:15 pm"]]]], "phone": "(951) 926-7201", "closed": false, "gPlusPlaceId": "104699454385822125632", "gps": [33.703804, -117.003209]}
{"name": "Blue Ribbon Cleaners", "price": null, "address": ["Parole", "Annapolis, MD"], "hours": null, "phone": "(410) 266-6123", "closed": false, "gPlusPlaceId": "103054478949000078829", "gps": [38.979759, -76.547538]}
{"name": "Portofino", "price": null, "address": ["\u0443\u043b. \u0422\u0443\u0442\u0430\u0435\u0432\u0430, 1", "Nazran, Ingushetia, Russia", "366720"], "hours": [["Monday", [["9:30 am--9:00 pm"]]], ["Tuesday", [["9:30 am--9:00 pm"]]], ["Wednesday", [["9:30 am--9:00 pm"]], 1], ["Thu

In [13]:
with open('first13lines.jsonl', 'r') as testfile:
    line = next(testfile)
    
normalised_line = json.dumps(ast.literal_eval(line))

In [16]:
normalised_line = json.loads(normalised_line)
normalised_line

{'name': 'Diamond Valley Lake Marina',
 'price': None,
 'address': ['2615 Angler Ave', 'Hemet, CA 92545'],
 'hours': [['Monday', [['6:30 am--4:15 pm']]],
  ['Tuesday', [['6:30 am--4:15 pm']]],
  ['Wednesday', [['6:30 am--4:15 pm']], 1],
  ['Thursday', [['6:30 am--4:15 pm']]],
  ['Friday', [['6:30 am--4:15 pm']]],
  ['Saturday', [['6:30 am--4:15 pm']]],
  ['Sunday', [['6:30 am--4:15 pm']]]],
 'phone': '(951) 926-7201',
 'closed': False,
 'gPlusPlaceId': '104699454385822125632',
 'gps': [33.703804, -117.003209]}

In [19]:
print(normalised_line.keys())

dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])


list

In [24]:
# Test the format of hours
hours = normalised_line['hours']
#  The nesting seems arbitrary.
hours[0][1][0][0]

'6:30 am--4:15 pm'

Testing GeoText for city and country detection

In [61]:
# Testing GeoText for city and country detection
with open('first13lines.jsonl', 'r') as testfile:
    for line in testfile:
        normalised_line = json.dumps(ast.literal_eval(line))
        address = json.loads(normalised_line).get('address')
        address = ', '.join(address)
        print(address)
        address = GeoText(address)
        print(address.cities)
        print(address.countries)
        print(address.country_mentions)
        print(address.nationalities)

2615 Angler Ave, Hemet, CA 92545
['Hemet']
[]
OrderedDict([('US', 1)])
[]
Parole, Annapolis, MD
['Parole', 'Annapolis']
[]
OrderedDict([('US', 2)])
[]
ул. Тутаева, 1, Nazran, Ingushetia, Russia, 366720
[]
['Russia']
OrderedDict([('RU', 1)])
[]
5322 W 26th St, Sioux Falls, SD 57106
['Sioux Falls']
[]
OrderedDict([('US', 1)])
[]
Jl. Angkatan 45, Kompleks Palembang Square, South Sumatra 30137, Indonesia
[]
['Indonesia']
OrderedDict([('ID', 1)])
[]
Stuttgarter Straße 44, 75378 Bad Liebenzell, Germany
[]
['Germany']
OrderedDict([('DE', 1)])
[]
2033 W Oklahoma Ave, Ulysses, KS 67880
[]
[]
OrderedDict()
[]
Jalan Demak, Petra Jaya, Kuching, Sarawak, Malaysia
['Kuching']
['Malaysia']
OrderedDict([('MY', 2)])
[]
208 Yumotochaya, Hakone, Ashigarashimo District, Kanagawa 250-0312, Japan
[]
['Japan']
OrderedDict([('JP', 1)])
[]
17960 NW Evergreen Pkwy, Beaverton, OR 97006
['Beaverton']
[]
OrderedDict([('US', 1)])
[]
3718 Wilmington Pike, Dayton, OH 45429
['Dayton']
[]
OrderedDict([('US', 1)])
[]
15

As we can see, the addresses are of different formats. Sometimes, the country is the last element of the list, sometimes the pincode is the last element. Annoyingly, the USA records don't have a country. But luckily, we won't be touching the USA records with a bargepole. 

GeoText only works with title case!! The .title() is therefore necessary, as below.

In [64]:
add = ', '.join(['208 Yumotochaya', 'Hakone, Ashigarashimo District, Kanagawa 250-0312', 'japan']).title()
add = GeoText(add)
add.countries

['Japan']

Our hotels are located in 6 countries: France, Italy, Spain, United Kingdom, Austria, Netherlands. Let's restrict the records to these countries.

In [27]:
countries = ['france', 'italy', 'spain', 'united kingdom', 'austria', 'netherlands']
with open('first13lines.jsonl', 'r') as testfile:
    for line in testfile:
        # dumps puts the literally evaluated string (double quotes etc.) in a json formatted string
        # We convert to a json formatted dict by loading this string with json.loads.
        normalised_dict = json.loads(json.dumps(ast.literal_eval(line)))
        print(normalised_dict.keys())
        joined_address = ', '.join(normalised_dict.get('address'))
        if any(country in joined_address.lower() for country in countries):
            # Country found in the list, find which country it is.
            # GeoText module only detects the city/country if it is capitalised
            geo_address = GeoText(joined_address.title())
            try:
                matching_country = geo_address.countries[0]
            except IndexError:
                matching_country = [country for country in countries if country in joined_address.lower()][0].title()
            try:
                matching_city = geo_address.cities[0]
            except IndexError:
                matching_city = None
            normalised_dict['country'] = matching_country
            normalised_dict['city'] = matching_city
            normalised_dict['latitude'] = normalised_dict.get('gps')[0]
            normalised_dict['longitude'] = normalised_dict.get('gps')[1]
            del normalised_dict['closed']
            del normalised_dict['gps']
            

dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])

In [33]:
hours = [['Monday', [['6:30 am--4:15 pm']]],
  ['Tuesday', [['6:30 am--4:15 pm']]],
  ['Wednesday', [['6:30 am--4:15 pm']], 1],
  ['Thursday', [['6:30 am--4:15 pm']]],
  ['Friday', [['6:30 am--4:15 pm']]],
  ['Saturday', [['6:30 am--4:15 pm']]],
  ['Sunday', [['6:30 am--4:15 pm']]]]

from itertools import chain
list(chain.from_iterable(hours))

['Monday',
 [['6:30 am--4:15 pm']],
 'Tuesday',
 [['6:30 am--4:15 pm']],
 'Wednesday',
 [['6:30 am--4:15 pm']],
 1,
 'Thursday',
 [['6:30 am--4:15 pm']],
 'Friday',
 [['6:30 am--4:15 pm']],
 'Saturday',
 [['6:30 am--4:15 pm']],
 'Sunday',
 [['6:30 am--4:15 pm']]]

In [44]:
import collections
def flatten(day_hours_list):
    for el in day_hours_list:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            print('she', el)
            yield from flatten(el)
        else:
            print('he', el, type(el))
            yield el
print(list(flatten([['sd', 2, ['sf', 4]]])))

def process_hours(hours):
    if hours is None:
        # Return None for each day
        return [None] * 7
    return_dict = dict()
    for day_hours in hours:
        return_dict[day_hours[0]] = flatten()
        

she ['sd', 2, ['sf', 4]]
he sd <class 'str'>
he 2 <class 'int'>
she ['sf', 4]
he sf <class 'str'>
he 4 <class 'int'>
['sd', 2, 'sf', 4]


In [50]:
def flatten(day_hours_list):
    """ Generator which flattens a list of lists recursively by yielding strings and ints/floats
    directly and recursively calling the generator func if it's an iterable """
    for el in day_hours_list:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

def process_hours(hours):
    """ Takes a Json array of hours in the following form (None if not present), and returns
    a dictionary containing days as keys and opening hours as string values
    [['Monday', [['6:30 am--4:15 pm']]],
    ['Tuesday', [['6:30 am--4:15 pm']]],
    ['Wednesday', [['6:30 am--4:15 pm']], 1],
    ['Thursday', [['6:30 am--4:15 pm']]],
    ['Friday', [['6:30 am--4:15 pm']]],
    ['Saturday', [['6:30 am--4:15 pm']]],
    ['Sunday', [['6:30 am--4:15 pm']]]] """
    keys = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    if hours is None:
        # Return None for each day
        return {k:None for k in keys}
    return_dict = dict()
    for day_hours in hours:
        return_dict[day_hours[0]] = next(flatten(day_hours[1]))
        
    return return_dict        

In [51]:
hours = [['Monday', [['6:30 am--4:15 pm']]],
  ['Tuesday', [['6:30 am--4:15 pm']]],
  ['Wednesday', [['6:30 am--4:15 pm']], 1],
  ['Thursday', [['6:30 am--4:15 pm']]],
  ['Friday', [['6:30 am--4:15 pm']]],
  ['Saturday', [['6:30 am--4:15 pm']]],
  ['Sunday', [['6:30 am--4:15 pm']]]]
process_hours(hours)

{'Monday': '6:30 am--4:15 pm',
 'Tuesday': '6:30 am--4:15 pm',
 'Wednesday': '6:30 am--4:15 pm',
 'Thursday': '6:30 am--4:15 pm',
 'Friday': '6:30 am--4:15 pm',
 'Saturday': '6:30 am--4:15 pm',
 'Sunday': '6:30 am--4:15 pm'}

Now, it's time for the complete end-to-end program.

In [70]:
def flatten(day_hours_list):
    """ Generator which flattens a list of lists recursively by yielding strings and ints/floats
    directly and recursively calling the generator func if it's an iterable """
    for el in day_hours_list:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

def process_hours(opening_hours):
    """ Takes a Json array of opening_hours in the following form (None if not present), and
    returns a dictionary containing days as keys and opening hours as string values
    [['Monday', [['6:30 am--4:15 pm']]],
    ['Tuesday', [['6:30 am--4:15 pm']]],
    ['Wednesday', [['6:30 am--4:15 pm']], 1],
    ['Thursday', [['6:30 am--4:15 pm']]],
    ['Friday', [['6:30 am--4:15 pm']]],
    ['Saturday', [['6:30 am--4:15 pm']]],
    ['Sunday', [['6:30 am--4:15 pm']]]] """
    day_keys = ['MondayHours', 'TuesdayHours', 'WednesdayHours', 'ThursdayHours',
            'FridayHours', 'SaturdayHours', 'SundayHours']
    if opening_hours is None:
        # Return None for each day
        return {k:None for k in day_keys}
    return_dict = dict()
    for day_hours in opening_hours:
        # day_hours[0] is the day, [1] is the list of lists
        return_dict[day_hours[0]+'Hours'] = next(flatten(day_hours[1]))
        
    return return_dict   

countries = ['france', 'italy', 'spain', 'united kingdom', 'austria', 'netherlands']
with jsonlines.open('output_first13lines.jsonl', mode='w') as writer:
    with open('first13lines_edited.jsonl', 'r') as testfile:
        # fields: 'name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'
        for line in testfile:
            normalised_dict = ast.literal_eval(line)
            print(normalised_dict.keys())
            joined_address = ', '.join(normalised_dict.get('address'))
            if any(country in joined_address.lower() for country in countries):
                # Country found in the list, find which country it is.
                # GeoText module only detects the city/country if it is capitalised
                geo_address = GeoText(joined_address.title())
                try:
                    matching_country = geo_address.countries[0]
                except IndexError:
                    matching_country = [country for country in countries if country in joined_address.lower()][0].title()
                try:
                    #If it matches 2 cities for some reason, take the one closer to the end of the string.
                    matching_city = geo_address.cities[-1]
                except IndexError:
                    matching_city = None
                normalised_dict['country'] = matching_country
                normalised_dict['city'] = matching_city
                normalised_dict['latitude'] = normalised_dict.get('gps')[0]
                normalised_dict['longitude'] = normalised_dict.get('gps')[1]
                opening_hours_dict = process_hours(normalised_dict.get('hours'))
                # Add the new keys (Monday, Tuesday, ...)
                normalised_dict.update(opening_hours_dict)
                print(normalised_dict.keys())
                del normalised_dict['closed']
                del normalised_dict['gps']
                del normalised_dict['hours']
                
                # Write to output jsonl file
                writer.write(normalised_dict)


dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps', 'country', 'city', 'latitude', 'longitude', 'MondayHours', 'TuesdayHours', 'WednesdayHours', 'ThursdayHours', 'FridayHours', 'SaturdayHours', 'SundayHours'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps'])
dict_keys(['name', 'price', 'address', 'hours', 'phone', 'closed', 'gPlusPlaceId', 'gps', 'country', 'city', 'latit